# Import Libraries

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math

from os import listdir, walk
from os.path import isfile, join

# Load Predicted Data

In [24]:
%run activity_summary.ipynb
%run ../ddc/preprocessing.ipynb
%run ../ddc/load_dataset.ipynb

In [4]:
subj_range = np.hstack((np.arange(2001,2002),np.arange(3001,3006)))
all_patients = [str(i) for i in subj_range]

In [5]:
datapath = '../../DDC_Data/'
basepath = '../../'

In [6]:
csv_path = datapath + 'prediction/predicted_data_' + all_patients[0] + '_to_' + all_patients[-1] + '.csv'
df_all_p_sorted = pd.read_csv(csv_path)

In [7]:
df_date = df_all_p_sorted.copy()
df_date['date'] = df_date['timestamp'].apply(lambda x: x.split(' ')[0])
df_date['time'] = df_date['timestamp'].apply(lambda x: x.split(' ')[1])

cols = ['ID','date','time','x','y','z','HR','y_pred']
df_date = df_date[cols]

# Summarize Data

In [25]:
df_summary_all, df_act_period = get_df_summary_all(all_patients, df_date)

/Users/admin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


In [26]:
cols = ['ID', 'date', 'from', 'to', 'y_pred']
df_act_period = df_act_period[cols]
df_act_period = df_act_period.reset_index(drop=True)

df_summary_all = df_summary_all.reset_index(drop=True)

In [28]:
actual_from_all = []
for i in range(len(df_summary_all)):
    keep_start = -1
    
    for j in range(len(df_act_period)):
        floor_start = get_floor_start(df_act_period.loc[j, 'from'])
        
        if(df_act_period.loc[j, 'date']==df_summary_all.loc[i, 'date'] and 
           df_act_period.loc[j, 'ID']==df_summary_all.loc[i, 'ID']):

            if(floor_start>keep_start and 
               floor_start==calc_sec(df_summary_all.loc[i, 'from'])):
                
                actual_from_all.append(df_act_period.loc[j, 'from'])
                keep_start = calc_sec(df_act_period.loc[j, 'from'])

            elif(floor_start<=keep_start):
                break

In [50]:
actual_to_all = []
for i in range(len(df_summary_all)-1, -1, -1):
    keep_finish = calc_sec(df_summary_all.loc[i, 'to'])
    
    for j in range(len(df_act_period)-1, -1, -1):
        
        ceil_finish = get_ceil_finish(df_act_period.loc[j, 'to'])
        
        
        if(df_act_period.loc[j, 'date']==df_summary_all.loc[i, 'date'] and 
           df_act_period.loc[j, 'ID']==df_summary_all.loc[i, 'ID']):

            if(keep_finish==ceil_finish):
                
                actual_to_all.append(df_act_period.loc[j, 'to'])
                break

In [56]:
df_summary_all['from actual'] = pd.Series(actual_from_all)
df_summary_all['to actual'] = pd.Series(actual_to_all[::-1])

In [57]:
cols = ['ID', 'date', 'from', 'to', 'from actual', 'to actual', 
        'sit', 'sleep', 'stand', 'walk', 'total', 
        'sit count', 'sleep count', 'stand count', 'walk count', 
        'inactive count', 'active count', 'total count', 'transition count', 'duration per action']
df_summary_all = df_summary_all[cols]

In [52]:
csv_path = datapath + 'summary/all_day_summary_' + all_patients[0] + '_to_' + all_patients[-1] + '.csv'
df_summary_all.to_csv(csv_path)

In [53]:
csv_path = datapath + 'summary/activity_period_' + all_patients[0] + '_to_' + all_patients[-1] + '.csv'
df_act_period.to_csv(csv_path)